<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

**Описание проекта**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

**Цель проекта**:
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Ход выполнения проекта

1. Загрузим и изучим данные.
2. Ответим на вопрос и обоснуем решение. 
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 a. Изменится. Приведите примеры матриц.
 b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
4. Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

**Описание данных**

Набор данных находится в файле /datasets/insurance.csv. 

**Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:
    #локальная версия
    try:
        df = pd.read_csv('\datasets\insurance.csv')
    #сетевая версия
    except:
        df = pd.read_csv('/datasets/insurance.csv')
except:
    print('Не удалось прочесть файл. Возможно, был указан не верный путь.')  

In [3]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


В нашем распоряжении таблица размерностью 5000 строк на 5 столбцов.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Названия столбцов следует сменить на латинницу. Пропусков в данных не обнаружено, колонки "Возраст" и "Зарплата" имеют тип float64, возможно требуется изменить тип на int. Проверим это.

In [5]:
df.rename(columns={'Пол': 'gender',
                   'Возраст': 'age',
                   'Зарплата': 'salary',
                   'Члены семьи': 'family_members',
                   'Страховые выплаты': 'insurance_payments'
                  },
                    inplace=True
         )

In [6]:
df['age'] = df['age'].astype(int)
df['salary'] = df['salary'].astype(int)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              5000 non-null   int64
 1   age                 5000 non-null   int64
 2   salary              5000 non-null   int64
 3   family_members      5000 non-null   int64
 4   insurance_payments  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


In [8]:
df = df.drop_duplicates()

Мы удалили 153 явных дубликата.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              4847 non-null   int64
 1   age                 4847 non-null   int64
 2   salary              4847 non-null   int64
 3   family_members      4847 non-null   int64
 4   insurance_payments  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [10]:
df.sample(5)

,gender,age,salary,family_members,insurance_payments
2644,1,33,40000,1,0
4825,1,23,31500,0,0
312,1,33,43700,2,0
2040,0,20,14700,0,0
1622,0,37,54000,1,0


Мы привели названия столбцов в надлежащий вид, а также все столбцы теперь имеют верный тип.

In [11]:
df.describe()

,gender,age,salary,family_members,insurance_payments
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [12]:
df['family_members'].median()

1.0

In [13]:
df['insurance_payments'].sum()

738

In [14]:
df.shape

(4847, 5)

- среднее значение пола близко к 0.5, это означает, что в данных примерно равное количество мужчин и женщин.
- в базе клиенты в возрасте от 18 до 65 лет, средний возраст которых близок к 31 году.
- заработная плата варируется от 5300 условных единиц до 79000 условных единиц, средний заработок составляет 39916.36 условных единиц.
- у клиентов от 0 до 6 членов в семье, среднестатистический клиент имеет 1 члена семьи.
- проводилось 740 страховых выплат.

Данные готовы для дальнейшей обработки.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** R2 не изменится

**Обоснование:**

Умножение матрицы на матрицу — это умножение первой матрицы на все столбцы второй.

При матричном умножении по двум матрицам строится третья. Она состоит из скалярных произведений строк первой матрицы на столбцы второй. Так результатом произведения i-й строки матрицы A (Ai) на j-й столбец матрицы B (Bj) станет элемент матрицы С c индексами i, j (Cij):

$$
Сij = (Ai, Bj) 
$$

Умножение матрицы на матрицу возможно, если ширина первой матрицы А (𝑚×𝑛) равна высоте второй матрицы В (𝑛×r). Произведение A и B даст матрицу размера m×r. Размерность n «cхлопывается».

В нашем случае матрица признаков A размерностью (4847, 4) умножается на на обратимую произвольную матрицу P размерностью (4,4) в результате получаем новый (изменённый) набор данных - матрицу С с размернотью (4847, 4)

Каждый элеменет матрицы С c индексами i, j (Cij) будет суммой произведений каждого элемента строки первой матрицы на каждый элемент столбца второй матрицы.
Например значение элемента с i=1 и j=1 будет рассчитано так:

$$
С_{11}=A_{11}×P_{11}+A_{12}×P_{21}+A_{13}×P_{31}+A_{14}×P_{41}
$$

В Линейной регрессии предсказания вычисляются по формуле:
$$
a=Xw+w_0,
$$ 
где w0 - значение нулевого аргумента вектора w, которое предствляет собой величину сдвига модели, при подборе которого можно достигать более низкого значения среднеквадратичного отклонения MSE.

Упрощенная запись формулы предсказания Линейной регрессии 
$$
a=X_iw
$$
, где Xi - новые строки признаков


Подставим w в формулу:
$$
a=X_iw=X_i(X^TX)^{−1}X^Ty
$$

Упроситим формулу: $$((AB)^T=B^TA^T)$$


$$
a=X_i(X^TX)^{−1}X^Ty=X_iX^{−1}(X^T)^{−1}X^Ty=X_iX^{−1}y
$$

Домножим признаки на случайную матрицу P, в которой число строк равно чиcлу столбцов X и P - обратима, то есть из матрицы P
можно получить матрицу P^{−1}, при этом PP^{−1}=E, где E - единичная матрица.

Если обучить модель на измененных признаках, то модель найдёт новые коэффициенты w′.

$$
w′=((XP)^T(XP))^{−1}(XP)^Ty
$$

Подставим их в формулу предсказания и упростим выражение:

$$
a′=X_iPw′=X_iP((XP)^T(XP))^{−1}(XP)^Ty=X_iP(XP)^{−1}((XP)^T)^{−1}(XP)^Ty=X_iX^{−1}y
$$

Таким образом результат предсказания не изменится после преобразований, a = a′

<div class="alert-info">
1.
    
Мы берём
    
    
$$
w = \arg\min_w MSE(Xw, y)
$$

- это означает, что мы ищем значения параметров $w$, при которых среднеквадратичное отклонение ($MSE$) между прогнозируемыми значениями $Xw$ и реальными значениями y минимально.
  

и    
$$
w_P = \arg\min_w MSE(XPw_p, y)
$$  
    
Мы хотим найти соотношение между $w$ и $w_P$. Попробуем сделать замену $w = Pw_P$:

$$
MSE(XPw_P, y) = MSE(X(Pw_P), y)
$$

Так как $Xw = X(Pw_P) = XPw_P$, задача теперь выглядит как:

$$
\arg\min_w MSE(Xw, y) = \arg\min_w MSE(XPw_P, y)
$$

Таким образом, мы видим, что:

$$
w = Pw_P
$$

Таким образом, параметры линейной регрессии $w$ и $Pw_P$ совпадают.

</div>

<div class="alert-info">
Сначала раскроем под знаком транспонирования:
    
$$
w' = (P^T X^T X P)^{-1} (XP)^T y
$$
    
А теперь поставим скобочки чтобы лучше было видно: 
    
    
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$    
    

и видим 3 квадратные матрицы!
    
$$
P^T, (X^T X), P
$$  
    
Используем правило:
    
    
$$
(ABC)^{-1} = C^{-1}(AB)^{-1} = (BC)^{-1}A^{-1} = C^{-1}B^{-1}A^{-1}
$$
    
    
    
следовательно раскрываем так:

    
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$    
    
    
    -----------------------------------------------------------------------------------------------------------------------
    
Подставим их в формулу предсказания:

$$
a′=X_iPw′=X_i P P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y=X_i (X^T X)^{-1} X^T y=X_iw = a
$$
    
Таким образом результат предсказания не изменится после преобразований, $a = a′$

## Алгоритм преобразования

**Алгоритм**

Для корректной расшифровки данных требуется обратимая матрица, иначе данные не смогут быть расшифрованы.

1. Создадим квадратную матрицу (4,4), размерность соответствует количеству фичей.
2. Заполняем созданную матрицу случайными числами.
3. Проверяем матрицу на обратимость.
4. В случае, если матрица обратима умножаем признаки на эту матрицу.

**Обоснование**

Наша квадратная матрица является своеобразным ключом к зашифрованным данным. При утечке данных злоумышленники столкнутся с проблемой, т.к. этот ключ требует перебора значений, что дает данным дополнительную безопасность, т.к. без него данные будут расшифрованы некорректно. Предполагается, что при таком преобразовании качество предсказаний целевого признака линейной моделью не должно пострадать после шифрования данных.

## Проверка алгоритма

Сохраним в переменные обучающие признаки и целевой признак, выведем на экран размерность полученных переменных.

In [15]:
features = df.drop(["insurance_payments"], axis=1)

In [16]:
target = df["insurance_payments"]

In [17]:
features.shape, target.shape[0]

((4847, 4), 4847)

Создадим квадратную матрицу и заполним её случайными числами.

In [18]:
matrix = np.random.randint(1000,size = (4, 4))

In [19]:
matrix

array([[ 45, 875, 766, 801],
       [979,  77, 732, 898],
       [635, 220, 622, 539],
       [963, 477, 949, 295]])

Проверим на инвертируемость (обратимость) матрицу.

In [20]:
matrix_invert = np.linalg.inv(matrix)

In [21]:
matrix_invert

array([[ 0.00172561,  0.00994131, -0.0220933 ,  0.00541957],
       [ 0.00354909,  0.01145668, -0.02800218,  0.0066517 ],
       [-0.00382471, -0.0167668 ,  0.03785171, -0.00773524],
       [ 0.00093211,  0.00296056, -0.00436742, -0.00017344]])

Матрица обратима. Умножим матрицу признаков на сгенерированную квадратную матрицу.

Инициализируем модель и обучим её.

In [22]:
model = LinearRegression(normalize=True)
model.fit(features, target)
predictions = model.predict(features)

Рассчитаем значение R2.

In [23]:
r2 = r2_score(target, predictions)
print(f'R2_score на оригинальных данных: {round(r2, 5)}')

R2_score на оригинальных данных: 0.4302


Перемножим матрицу признаков и квадратную матрицу.

In [24]:
features_matrix = features.dot(matrix)

Инициализируем модель и обучим её на зашифрованных признаках. Предскажем целевой признак.

In [25]:
model = LinearRegression(normalize = True)
model.fit(features_matrix, target)
predictions = model.predict(features_matrix)

Рассчитаем значение R2.

In [26]:
r2 = r2_score(target, predictions)
print(f'R2_score на зашифрованных данных: {round(r2, 5)}')

R2_score на зашифрованных данных: 0.4302


**Итоговый вывод:**
    
Значения скоров R2 на оригинальных и зашифрованных данных имеют одинаковое значение, из чего следует, что данные пользователей защищены, при том, что модель не потеряла в качестве предсказания целевого признака.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования